In [ ]:
import pandas as pd
import time
from datetime import datetime

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Milestone II/cleaned_stock_data.csv")
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Company Name
0,2023-12-29,50.30,50.57,49.77,50.25,49.59,29266500,INTC,Intel
1,2023-12-28,50.81,50.87,50.16,50.39,49.72,27705200,INTC,Intel
2,2023-12-27,50.63,51.28,50.19,50.76,50.09,52148000,INTC,Intel
3,2023-12-26,48.92,50.52,48.71,50.50,49.83,60287400,INTC,Intel
4,2023-12-22,47.25,48.16,47.20,48.00,47.37,30053700,INTC,Intel


In [ ]:
# pip install pandas_ta

In [ ]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import pandas_ta as ta  # Ensure you have pandas_ta installed
from sklearn.preprocessing import StandardScaler

df['Date'] = pd.to_datetime(df['Date'])


# Group by 'Ticker' to handle multiple stocks in your dataset
grouped_df = df.groupby('Ticker')

# Garman-Klass Volatility Calculation without groupby apply
df['garman_klass_vol'] = (
    ((np.log(df['High']) - np.log(df['Low'])) ** 2) / 2
    - (2 * np.log(2) - 1) * ((np.log(df['Adj Close']) - np.log(df['Open'])) ** 2)
)

# Relative Strength Index (RSI)
df['rsi'] = df.groupby('Ticker')['Adj Close'].transform(lambda x: ta.rsi(close=x, length=20))

# Bollinger Bands (BB) - Lower, Mid, High
df['bb_low'] = df.groupby('Ticker')['Adj Close'].transform(lambda x: ta.bbands(close=np.log1p(x), length=20).iloc[:, 0])
df['bb_mid'] = df.groupby('Ticker')['Adj Close'].transform(lambda x: ta.bbands(close=np.log1p(x), length=20).iloc[:, 1])
df['bb_high'] = df.groupby('Ticker')['Adj Close'].transform(lambda x: ta.bbands(close=np.log1p(x), length=20).iloc[:, 2])

def compute_atr(stock_data):
    atr = ta.atr(high=stock_data['High'], low=stock_data['Low'], close=stock_data['Close'], length=14)
    return atr

# Average True Range (ATR)
df['atr'] = df.groupby('Ticker', group_keys=False).apply(lambda group: compute_atr(group)).reset_index(level=0, drop=True)

# Moving Average Convergence Divergence (MACD)
def compute_macd(close):
    macd = ta.macd(close=close, fast=12, slow=26, signal=9).iloc[:, 0]
    return macd

df['macd'] = df.groupby('Ticker', group_keys=False)['Adj Close'].apply(lambda x: compute_macd(x)).reset_index(level=0, drop=True)

# Dollar Volume
df['dollar_volume'] = (df['Adj Close'] * df['Volume']) / 1e6

# Resetting index if needed
df.reset_index(inplace=True, drop=True)

# Output for inspection
print(df.head())


df

        Date   Open   High    Low  Close  Adj Close    Volume Ticker  \
0 2023-12-29  50.30  50.57  49.77  50.25      49.59  29266500   INTC   
1 2023-12-28  50.81  50.87  50.16  50.39      49.72  27705200   INTC   
2 2023-12-27  50.63  51.28  50.19  50.76      50.09  52148000   INTC   
3 2023-12-26  48.92  50.52  48.71  50.50      49.83  60287400   INTC   
4 2023-12-22  47.25  48.16  47.20  48.00      47.37  30053700   INTC   

  Company Name  garman_klass_vol  rsi  bb_low  bb_mid  bb_high  atr  macd  \
0        Intel          0.000049  NaN     NaN     NaN      NaN  NaN   NaN   
1        Intel         -0.000083  NaN     NaN     NaN      NaN  NaN   NaN   
2        Intel          0.000186  NaN     NaN     NaN      NaN  NaN   NaN   
3        Intel          0.000534  NaN     NaN     NaN      NaN  NaN   NaN   
4        Intel          0.000200  NaN     NaN     NaN      NaN  NaN   NaN   

   dollar_volume  
0    1451.325735  
1    1377.502544  
2    2612.093320  
3    3004.121142  
4    1423

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Company Name,garman_klass_vol,rsi,bb_low,bb_mid,bb_high,atr,macd,dollar_volume
0,2023-12-29,50.30,50.57,49.77,50.25,49.59,29266500,INTC,Intel,0.000049,NaN,NaN,NaN,NaN,NaN,NaN,1451.325735
1,2023-12-28,50.81,50.87,50.16,50.39,49.72,27705200,INTC,Intel,-0.000083,NaN,NaN,NaN,NaN,NaN,NaN,1377.502544
2,2023-12-27,50.63,51.28,50.19,50.76,50.09,52148000,INTC,Intel,0.000186,NaN,NaN,NaN,NaN,NaN,NaN,2612.093320
3,2023-12-26,48.92,50.52,48.71,50.50,49.83,60287400,INTC,Intel,0.000534,NaN,NaN,NaN,NaN,NaN,NaN,3004.121142
4,2023-12-22,47.25,48.16,47.20,48.00,47.37,30053700,INTC,Intel,0.000200,NaN,NaN,NaN,NaN,NaN,NaN,1423.643769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186292,2023-09-20,55.17,55.40,51.52,52.91,52.91,16369500,ARM,Arm,0.001960,47.612357,3.941181,3.990420,4.039659,2.005104,-0.254211,866.110245
186293,2023-09-19,56.25,56.78,53.88,55.17,55.17,18308600,ARM,Arm,0.001229,51.715612,3.941651,3.993051,4.044451,2.257597,-0.057333,1010.085462
186294,2023-09-18,57.95,58.74,55.02,58.00,58.00,34571900,ARM,Arm,0.002140,56.234069,3.936124,3.998329,4.060533,2.311340,0.323325,2005.170200
186295,2023-09-15,68.63,69.00,60.75,60.75,60.75,74568900,ARM,Arm,0.002362,60.057404,3.928738,4.007133,4.085528,2.485530,0.837249,4530.060675


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder_ticker = LabelEncoder()
label_encoder_company = LabelEncoder()

# Fit and transform the 'Ticker' column
df['Ticker_numeric'] = label_encoder_ticker.fit_transform(df['Ticker'])

# Fit and transform the 'Company Name' column
df['Company_Name_numeric'] = label_encoder_company.fit_transform(df['Company Name'])

# Output for inspection
print(df.head())


        Date   Open   High    Low  Close  Adj Close    Volume Ticker  \
0 2023-12-29  50.30  50.57  49.77  50.25      49.59  29266500   INTC   
1 2023-12-28  50.81  50.87  50.16  50.39      49.72  27705200   INTC   
2 2023-12-27  50.63  51.28  50.19  50.76      50.09  52148000   INTC   
3 2023-12-26  48.92  50.52  48.71  50.50      49.83  60287400   INTC   
4 2023-12-22  47.25  48.16  47.20  48.00      47.37  30053700   INTC   

  Company Name  garman_klass_vol  rsi  bb_low  bb_mid  bb_high  atr  macd  \
0        Intel          0.000049  NaN     NaN     NaN      NaN  NaN   NaN   
1        Intel         -0.000083  NaN     NaN     NaN      NaN  NaN   NaN   
2        Intel          0.000186  NaN     NaN     NaN      NaN  NaN   NaN   
3        Intel          0.000534  NaN     NaN     NaN      NaN  NaN   NaN   
4        Intel          0.000200  NaN     NaN     NaN      NaN  NaN   NaN   

   dollar_volume  Ticker_numeric  Company_Name_numeric  
0    1451.325735              20               

In [ ]:
unique_rows = df[["Ticker", "Company Name", "Ticker_numeric", "Company_Name_numeric"]].drop_duplicates().sort_values(by="Company Name")
unique_rows.to_csv("/content/drive/MyDrive/Milestone II/Ticker_Name_to_Numeric.csv", index = False)

In [ ]:
top_10_nasdaq = ["Apple", "Microsoft", "NVIDIA", "Broadcom", "Meta Platforms", "Alphabet A",
                 "AMD", "Adobe", "Qualcomm", "Texas Instruments"]

In [ ]:
df_top_10 = df[df["Company Name"].isin(top_10_nasdaq)]
df_top_10.drop(['Ticker', 'Company Name'], axis=1, inplace=True)
df_top_10.to_csv("/content/drive/MyDrive/Milestone II/Top_10_NASDAQ.csv", index = False)

In [ ]:
df.drop(['Ticker', 'Company Name'], axis=1, inplace=True)
df.to_csv('/content/drive/MyDrive/Milestone II/feature_eng_stock.csv', index=False)